# PART 0: INSTALL (for Colab and New Devices)

In [2]:
!pip install cma

     |████████████████████████████████| 230 kB 1.4 MB/s eta 0:00:01


# PART 1: EVALUATION

In [4]:
import cma
from cma.fitness_transformations import EvalParallel2

## TOY EXAMPLE: ROSENBROCK FUNCTION

In [5]:
es = cma.CMAEvolutionStrategy(8 * [0], 0.5)

f = cma.ff.rosen
#es.optimize(f)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 8 (seed=130092, Tue Nov 10 20:57:39 2020)


In [10]:
with EvalParallel2(f, number_of_processes=12) as eval:
    while not es.stop():
        X = es.ask()
        es.tell(X, eval(X))
print(es.result[1], es.result[2])

5.488439864559804e-15 4000


## SLIGHTLY BIGGER TOY EXAMPLE: MNIST
Builds an 1-layer neural network of 

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# "GLOBALS"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TRAIN_LOADER = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

TEST_LOADER = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
BOUNDS = np.array([[-6, 2,  2,  2,  2,  2,  16],
                   [0,  10, 20, 20, 10, 10, 512]])

def mnist_run(hyperparameters):
    """
        Generate a training run for an MNIST classifier with given hyperperameter values
        :param hyperparameters (3, int): 1D array of hyperparameter values
        :return: evaluation metric
    """
    LR = hyperparameters[0] #LEARNING RATE (EXPONENT)
    EP = hyperparameters[1] #EPOCHS
    C1 = hyperparameters[2] #CHANNEL SIZE 1
    C2 = hyperparameters[3] #CHANNEL SIZE 2
    K1 = hyperparameters[4] #KERNEL SIZE 1
    K2 = hyperparameters[5] #KERNEL SIZE 2
    LW = hyperparameters[6] #LINEAR LAYER WIDTH
    
    class MNIST_Net(nn.Module):
        def __init__(self, C1, K1, C2, K2, LW):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, C1, kernel_size=K1)
            w1 = (28-K1)+1
            self.conv2 = nn.Conv2d(C1, C2, kernel_size=K2)
            w2 = (w1-K2)+1
            
            self.width2 = w2*w2*C2
            self.fc1 = nn.Linear(self.width2, LW)
            self.fc2 = nn.Linear(LW, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x = x.view(-1, self.width2)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x)
    
    def train(EP):
        network.train()
        for i in range(EP):
            for batch_idx, (data, target) in enumerate(train_loader):
                optimizer.zero_grad()
                output = network(data)
                loss = F.nll_loss(output, target)
                loss.backward()
                optimizer.step()
            
    def test():
        network.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = network(data)
                test_loss += F.nll_loss(output, target, size_average=False).item()
                pred = output.data.max(1, keepdim=True)[1]
                correct += pred.eq(target.data.view_as(pred)).sum()
        return correct
    
    network = MNIST_Net()
    network.to(DEVICE)
    optimizer = optim.SGD(network.parameters(), lr=1**LR, momentum=0.9)

In [ ]:
options = cma.CMAOptions()
options.set('integer_variables', [0, 1, 2, 3, 4, 5, 6])

f = cma.BoundPenalty(mnist_run, bounds=BOUNDS)

es = cma.CMAEvolutionStrategy([-3, 5, 10, 10, 5, 5, 50], [1, 1, 2, 2, 1, 1, 32])

with EvalParallel2(f, number_of_processes=1) as eval:
    while not es.stop():
        X = es.ask()
        es.tell(X, eval(X))
print(es.result[1], es.result[2])

# PART 2: EVAL FUNCTION WRAPPER

In [ ]:
def launch_run(hyperparameters):
    """
        TODO: write training run code for ASR
        TODO: integrate with ESPnet and feature extraction model training procedures

        Generate a training run of ASR system with selected hyperparameters
        :param hyperparameters: 1D array of hyperparameter values
        :return: evaluation metric
    """  
    raise NotImplementedError